## Alcholic Anonymous News articles extraction from The Hindu Newspaper. Data Extraction of following parameters

- Headline
- Description
- Author
- Published_Date
- News
- URL


### Importing the necessary Libraries

In [1]:
from bs4 import BeautifulSoup # Python library for pulling data out of HTML and XML files
from requests import get # standard for making HTTP requests in Python
import pandas as pd # library written for data manipulation and analysis
import sys #  System-specific parameters and functions.

### Creating Empty lists for Alcholic Anonymous News Articles parameters data to be extracted

In [2]:
headlines, descriptions, dates, authors, news, urls = [], [], [], [], [], []

### Finding the total no.of.pages by total no.of articles from The Hindu search results

In [3]:
url = 'https://www.thehindu.com/search/?q=Alcoholic%20Anonymous&order=DESC&sort=publishdate'
soup = BeautifulSoup(get(url).text, 'lxml')

# for collecting the total number of article
article_count = int(''.join(i for i in soup.select_one('.section-controls').span.text.split('of')[1] if i.isdigit()))

# to find the no.of pages
max_pages = int((int(article_count)//12) + 2)
max_pages

988

### Iterates max_pages value through for loop. Scraping the Articles urls along with basic details which comes handly as follows  
- Headline
- Author
- Published_Date
- URL

In [4]:
%%time
for index, i in enumerate(range(1, max_pages)):    
    url = 'https://www.thehindu.com/search/?q=Alcoholic%20Anonymous&order=DESC&sort=publishdate&page=' + str(i)
    soup = BeautifulSoup(get(url).text, 'lxml')
    
    # Extracts url's 
    url = [soup.select('a.story-card75x1-text')[i]['href'] for i in range(len(soup.select('a.story-card75x1-text')))]
    urls.extend(url)
    
    # Extracts headline's
    headline = [soup.select('.story-card75x1-text')[i].text.strip() for i in range(len(soup.select('.section-name')))]
    headlines.extend(headline)
    
    # Extracts author's 
    for i in range(len(soup.select('.story-card-news'))):
        try:
            authors.append(soup.select('.story-card-news')[i].select_one('.story-card-33-author-name').text.strip())
        except:
            authors.append(None)
    
    # Extracts Date's
    date = [soup.select('.dateline')[i].text.strip() for i in range(len(soup.select('.dateline')))]
    dates.extend(date)
    
    sys.stdout.write('\r' + str(index) + ' : ' + str(max_pages))
    sys.stdout.flush()

986 : 988Wall time: 26min 12s


### Iterates urls through for loop. Scraping the Articles using BeautifulSoup4 which extracts the followings

- Description
- News

In [5]:
%%time
for index, url in enumerate(urls):
    try:
        # Parse the url to NewsPlease 
        soup = BeautifulSoup(get(url).text, 'lxml')
            
        try:
            # Extracts the Descriptions
            desc = soup.select_one('meta[name="description"]')['content'].strip().replace('\n', ' ')
            descriptions.append(desc)
        except:
            descriptions.append(None)
            
        try:
            # Extracts the news articles
            para = soup.select_one('.article').select('p')
            text = ''.join(str(e.text.strip().replace('\n', '').replace("[^a-zA-Z]", " ")) for e in para)
            news.append(text)
        except:
            news.append(None)
            
    except:
        descriptions.append(None)
        news.append(None)

    sys.stdout.write('\r' + str(index) + ' : ' + str(url) + '\r')
    sys.stdout.flush()

Wall time: 5h 2min 5shehindu.com/todays-paper/tp-miscellaneous/in-madurai-today/article3233838.ece3235280.ece46450.ece9.ecece/article3169273.ece7.ece569.eceeceps-have-turned-life-around-for-many-troubled-young-people/article8509546.ece


### Checking Array Length of each list to create DataFrame

In [6]:
print(len(headlines), len(descriptions), len(authors), len(dates), len(news), len(urls))

11842 11842 11842 11842 11842 11842


### Creating a csv file after validating  array length

In [7]:
tbl = pd.DataFrame({'Headlines' : headlines,
                    'Descriptions' : descriptions,
                    'Authors' : authors,
                    'Published_Dates' : dates, 
                    'Articles' : news,
                    'Source_URLs' : urls})

tbl.to_csv('The_Hindu.csv', index=False)